In [1]:
from bokeh.models import Legend
from bokeh.layouts import layout
from bokeh.palettes import Spectral6, OrRd3, GnBu3
from bokeh.transform import linear_cmap
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, BoxSelectTool, BoxZoomTool, UndoTool, ResetTool, PanTool, WheelZoomTool
from bokeh.io import output_notebook, output_file
from bokeh.models import ColumnDataSource, CustomJS, Slider, Range, Label, LabelSet, Range1d, Select, CDSView, IndexFilter
from bokeh.models import Div, RangeSlider, Spinner, MultiChoice

In [2]:
import pandas as pd

In [3]:
from bokeh.io import output_notebook, output_file

In [4]:
# from bokeh.models import MultiChoice


# def plot_chr(df_chr):



file = 'test_input.csv'
df = pd.read_csv(file)
# p = plot_chr(p, df, 'chr1')
# plot_chr(df)

# output_file('test_out.html')
# df = df_chr.copy()

Loading BokehJS ...

In [5]:
df.head()

,chr,Position,copyNum
0,chr1,4350001,1.963191
1,chr1,4800001,1.992764
2,chr1,5250001,1.992764
3,chr1,5850001,1.960534
4,chr1,6900001,1.958227


In [6]:
df['chr_int'] = pd.to_numeric(df['chr'].str.lstrip('chr'), errors='coerce')
df.loc[df['chr']=='chrX', 'chr_int'] = 23
df.loc[df['chr']=='chrY', 'chr_int'] = 24
# df.loc[df['chr_int'].isna(), 'chr_int'] = df.loc[df['chr_int'].isna(), 'chr'].str.lstrip('chr')
df = df.sort_values(by=['chr_int', 'Position'], ascending=True)
df_first_pos = df.drop_duplicates(subset='chr_int', keep='first') 

In [65]:
mapper1 = linear_cmap(field_name='copyNum', palette=OrRd3+ tuple(reversed(OrRd3)), 
                      low=1 ,high=3)
mapper2 = linear_cmap(field_name='copyNum', palette=GnBu3 + tuple(reversed(GnBu3)),
                      low=1 ,high=3)

In [7]:
df['color'] = linear_cmap(df['copyNum'], palette=OrRd3+ tuple(reversed(OrRd3)), low=1 ,high=3)['field']
mask = (df['chr_int'] % 2 == 1)
df.loc[mask, 'color'] = linear_cmap(df.loc[mask, 'copyNum'], palette=GnBu3 + tuple(reversed(GnBu3)), low=1 ,high=3)['field']

In [32]:
df_first_pos.head()

,chr,Position,copyNum,chr_int
0,chr1,4350001,1.963191,1.0
2247,chr2,150001,1.826206,2.0
2988,chr3,450001,2.614825,3.0
3406,chr4,1050001,2.165504,4.0
3803,chr5,1800001,2.186356,5.0


In [21]:
source = ColumnDataSource(df.reset_index())

In [95]:
output_notebook()
output_file('test_out.html')
p = figure(y_range=(0, 5), width=1200, height=300, 
           tools=[PanTool(dimensions="width"), 
                  BoxZoomTool(dimensions="width"), 
                  WheelZoomTool(dimensions="width"), 
                  ResetTool(), HoverTool(), UndoTool()],
          )


spinner = Spinner(title="Circle size",low=0.5,high=10,step=0.5,value=3,width=200,)
pc = p.circle(x='index', y='copyNum', color='red', fill_alpha=0.5, size=3, line_width=0, source=source)
spinner.js_link("value", pc.glyph, "size")
p.x_range.start = 0
div = Div(text='result')
p.x_range.js_on_change('start',
    CustomJS(args=dict(div=div),
             code="""
             var start = cb_obj.start
             var end = cb_obj.end
             div['text'] = "start: " + start.toFixed(0) + "   end: " + end.toFixed(0)
             """
    )
)

data = ColumnDataSource(df_first_pos.reset_index())
labels = LabelSet(x='index', y=4, text='chr', x_offset=5, y_offset=10, source=data, render_mode='canvas')
p.add_layout(labels)

OPTIONS = list(df_first_pos['chr'].unique())
multi_choice = MultiChoice(value=OPTIONS, options=OPTIONS, width=p.width, placeholder='choose a chromosome')
# multi_choice.on_change('value', multichoice_callback)
multi_choice.js_on_change("value", CustomJS(args=dict(source=source),
                                            code="""
    
    const text = this.value.toString();
    const args = text.split(",");
    console.log(args)
    //console.log(source.data)
    //console.log('multi_select: value=' + this.value, this.toString())
    //console.log(source)
    const indices = [];
    for (let i = 0; i < source.get_length(); i++){
        if (source.data['chr'][i] in args){
            indices.push(true);
        } else {
            indices.push(false);
        }
    }
    source = indices
    source.change.emit();
    console.log(indices)
"""))

# plot = layout([[spinner],[multi_choice], [p],])
plot = column(spinner, div, multi_choice, p)
show(plot)

Loading BokehJS ...

Start : This command cannot be run due to the error: The system cannot find the file specified.
At line:1 char:1
+ Start "file:///mnt/d/Yangk/GermountX/git/CNVcheck/test_out.html"
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 


In [98]:
source.remove()

/home/yangk/anaconda3/envs/GermountX/lib/python3.10/site-packages/bokeh/models/sources.py:408: UserWarning: Unable to find column '1' in data source
  warnings.warn("Unable to find column '%s' in data source" % name)


In [90]:
source.data['chr'] in ['chr1', 'chr2']

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [58]:
mapper1 = linear_cmap(field_name='copyNum', palette=OrRd3+ tuple(reversed(OrRd3)), 
                      low=1 ,high=3)
mapper2 = linear_cmap(field_name='copyNum', palette=GnBu3 + tuple(reversed(GnBu3)),
                      low=1 ,high=3)
spinner = Spinner(
    title="Circle size",  # a string to display above the widget
    low=0.5,  # the lowest possible number to pick
    high=10,  # the highest possible number to pick
    step=0.5,  # the increments by which the number can be adjusted
    value=3,  # the initial value to display in the widget
    width=200,  #  the width of the widget in pixels
)

chrs_int = df['chr_int'].unique()
chrs = df['chr'].unique()
end_point = 0
df['Position_axis'] = end_point
for c, ch in zip(chrs_int, chrs):
    # ch = f'chr{c}'
    # if  (c == 'X') or (c % 2):
    if type(c) == float and int(c) % 2:
        mapper = mapper1
    elif c == 'X':
        mapper = mapper1
    else:
        mapper = mapper2

    # if ch  in df['chr'].unique():
    df.loc[df['chr']==ch, 'Position_axis'] = df.loc[df['chr']==ch, 'Position'] + end_point
    end_point = df['Position_axis'].max()
    data = ColumnDataSource(df.loc[df['chr']==ch])
    pc = p.circle(x='Position_axis', y='copyNum', color=mapper, fill_alpha=0.5, size=3, line_width=0, source=data)#, legend_label=c)
    # legend_item.append((ch, [pc]))
    spinner.js_link("value", pc.glyph, "size")

OPTIONS = list(chrs)
multi_choice = MultiChoice(value=OPTIONS, options=OPTIONS, width=p.width, placeholder='choose a chromosome')
# multi_choice.on_change('value', multichoice_callback)
multi_choice.js_on_change("value", CustomJS(args=dict(div=div),
                                            code="""
    console.log('multi_select: value=' + this.value, this.toString())
"""))

#     show(multi_choice)

df_first_pos = df.drop_duplicates(subset='chr_int', keep='first')
data = ColumnDataSource(df_first_pos)
labels = LabelSet(x='Position_axis', y=4, text='chr',
          x_offset=5, y_offset=10, source=data, render_mode='canvas')

p.toolbar.logo = None
p.add_layout(labels)
# p.add_layout(legend, 'above')    
# plot = layout([[spinner],[range_slider],[p],])
# plot = layout([[spinner],[p],])
plot = layout([[spinner],[multi_choice], [p],])
show(plot)

    
    

In [141]:
def set_plot():
        p = figure(y_range=(0, 5), width=1200, height=300, 
               tools=[PanTool(dimensions="width"), 
                      BoxZoomTool(dimensions="width"), 
                      WheelZoomTool(dimensions="width"), 
                      ResetTool(), HoverTool(), UndoTool()],
              )
        return p
    
def plot_chr(p, df_chr, chrs=None):
    if chrs:
        if type(chrs)==str and chrs in df_chr['chr'].unique():
            df = df_chr.loc[df_chr['chr']==chrs,:].copy()
        if type(chrs)==list and len(chrs)>=1:
            df = df_chr.loc[df_chr['chr'].isin(chrs),:].copy()
    else:
        df = df_chr.copy()
        
    df['chr_int'] = pd.to_numeric(df['chr'].str.lstrip('chr'), errors='coerce').round(0)
    df.loc[df['chr_int'].isna(), 'chr_int'] = df.loc[df['chr_int'].isna(), 'chr'].str.lstrip('chr')
    df = df.sort_values(by=['chr_int', 'Position'], ascending=True)
    
    mapper1 = linear_cmap(field_name='copyNum', palette=OrRd3+ tuple(reversed(OrRd3)), low=1 ,high=3)
    mapper2 = linear_cmap(field_name='copyNum', palette=GnBu3 + tuple(reversed(GnBu3)),low=1 ,high=3)
    
    chrs_int = df['chr_int'].unique()
    chrs = df['chr'].unique()
    end_point = 0
    df['Position_axis'] = end_point
    for c, ch in zip(chrs_int, chrs):
        print(c,ch)
        if (type(c)==int or type(c)==float) and int(c) % 2:
            mapper = mapper1
        elif c == 'X':
            mapper = mapper1
        else:
            mapper = mapper2
            
        df.loc[df['chr']==ch, 'Position_axis'] = df.loc[df['chr']==ch, 'Position'] + end_point
        end_point = df['Position_axis'].max()
        data = ColumnDataSource(df.loc[df['chr']==ch])
        pc = p.circle(x='Position_axis', y='copyNum', color=mapper, fill_alpha=0.8, size=3, line_width=0, source=data)#, legend_label=c)
    
    df_first_pos = df.drop_duplicates(subset='chr_int', keep='first')
    data = ColumnDataSource(df_first_pos)
    labels = LabelSet(x='Position_axis', y=4, text='chr',x_offset=5, y_offset=10, source=data, render_mode='canvas')
    p.add_layout(labels)
    return p


def multi_choice_wiget(p, lst):
    OPTIONS = list(lst)
    multi_choice = MultiChoice(value=OPTIONS, options=OPTIONS, width=p.width, placeholder='choose a chromosome')
    multi_choice.js_on_change("value", CustomJS(code="""
        console.log('multi_choice: value=' + this.value, 
        this.toString())
    """))
    plot = layout([[multi_choice],[p]])
    
    return plot

    
def multichoice_callback(attr, old, new):
    p = plot_chr(p, df, 'chr1')
#     CustomJS(code="""
#         console.log('multi_choice: test value=' + this.value, 
#         this.toString())
#     """)
#     plot_chr(p, df_chr, chrs=new.split(','))
#     print("Previous label: " + old)
#     print("Updated label: " + new)

In [142]:



file = 'test_input.csv'
df = pd.read_csv(file)
p = set_plot()
# p = plot_chr(p, df, 'chr1')
p = multi_choice_wiget(p, ['chr1', 'chr2'])

In [ ]:
show(p)

In [40]:
df['chr_int'] = pd.to_numeric(df['chr'].str.lstrip('chr'), errors='coerce')
df.loc[df['chr_int'].isna(), 'chr_int'] = df.loc[df['chr_int'].isna(), 'chr'].str.lstrip('chr')
df = df.sort_values(by=['chr_int', 'Position'], ascending=True)

df_first_pos = df.drop_duplicates(subset='chr_int', keep='first')

In [16]:
import numpy as np
from bokeh.layouts import row, column
from bokeh.models import BoxAnnotation, CustomJS, Div
from bokeh.plotting import figure, output_file, show
from bokeh.models import MultiChoice

# output_file('range_update_callback.html')
output_notebook()
N = 4000

x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 1.5
colors = [
    "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
]

div = Div(width=400, text='xxx')
OPTIONS = ["foo", "bar", "baz", "quux"]

multi_choice = MultiChoice(value=["foo", "baz"], options=OPTIONS)
multi_choice.js_on_change("value", CustomJS(code="""
    console.log('multi_choice: value=' + this.value, this.toString())
"""))

p1 = figure(title='Pan and Zoom Here', x_range=(0, 100), y_range=(0, 100))
p1.scatter(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)

p1.x_range.js_on_change('start',
    CustomJS(args=dict(div=div),
             code="""
             var num = cb_obj.start
             div['text'] = num + ""
             console.log(num)
             
             """
    )
)


layout = column(multi_choice, row(div, p1))

show(layout)
# save(layout)

Loading BokehJS ...

In [10]:
p1.x_range.start

0

In [25]:
import numpy as np

from bokeh.layouts import row
from bokeh.models import BoxAnnotation, CustomJS
from bokeh.plotting import figure, output_file, show

# output_file('range_update_callback.html')
output_notebook()

N = 4000

x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 1.5
colors = [
    "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
]

box = BoxAnnotation(left=0, right=0, bottom=0, top=0,
    fill_alpha=0.1, line_color='black', fill_color='black')

jscode = """
    box[%r] = cb_obj.start
    box[%r] = 100
    box['bottom'] = 20
    box['top'] = 80
"""

p1 = figure(title='Pan and Zoom Here', x_range=(0, 100), y_range=(0, 100),
            tools='box_zoom,wheel_zoom,pan,reset', width=400, height=400)
p1.scatter(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)

xcb = CustomJS(args=dict(box=box), code=jscode % ('left', 'right'))
# ycb = CustomJS(args=dict(box=box), code=jscode % ('bottom', 'top'))

p1.x_range.js_on_change('start', xcb)
# p1.x_range.js_on_change('end', xcb)
# p1.y_range.js_on_change('start', ycb)
# p1.y_range.js_on_change('end', ycb)

p2 = figure(title='See Zoom Window Here', x_range=(0, 100), y_range=(0, 100),
            tools='', width=400, height=400)
p2.scatter(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)
p2.add_layout(box)

layout = row(p1, p2)

show(layout)

Loading BokehJS ...